지난 포스트에서 `stock_market.csv`의 몇 칼럼에서 null값(NaN)이 발견되었습니다.  
오늘은 결측치(missing value)들을 몇가지 방법으로 처리해 보겠습니다.  

결측치를 처리하는 방법은 크게 두가지로 나눌 수 있습니다.
- 결측치 제거
- 결측치 대체

하지만 그 전에, 먼저 데이터를 살펴봐야 결측치 처리 방향을 정할 수 있겠죠?

In [25]:
# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['axes.unicode_minus'] = False  # matplotlib 마이너스기호 표시
plt.rc('font', family='NanumGothic')  # matplotlib 한글폰트 표시

# pandas
import pandas as pd

# data
df = pd.read_csv('stock_market.csv')

# 1. 결측여부 확인

먼저 지난번 사용했던 방법으로 다시 전체 칼럼의 상태를 살펴보겠습니다.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    250 non-null    object 
 1   Name                  250 non-null    object 
 2   Market                250 non-null    object 
 3   Category              250 non-null    object 
 4   Capital               250 non-null    int64  
 5   PER                   249 non-null    float64
 6   EPS                   250 non-null    int64  
 7   ROE                   249 non-null    float64
 8   PBR                   250 non-null    float64
 9   BPS                   250 non-null    int64  
 10  Group_PER             250 non-null    float64
 11  Revenue               250 non-null    int64  
 12  Operating_Income      250 non-null    int64  
 13  Net_Income            250 non-null    int64  
 14  Dividend              189 non-null    float64
 15  Debt                  2

전체 행이 250개인 df에서 `PER`등이 일부 null값을 가지는 것을 확인할 수 있습니다.  

그리고 `isnull()`과 `sum()`을 함께쓰면 어느 칼럼에 몇개의 null이 있는지 좀 더 간단하게 볼 수 있습니다.

In [27]:
df.isnull().sum()  # df.isna().sum()

ID                       0
Name                     0
Market                   0
Category                 0
Capital                  0
PER                      1
EPS                      0
ROE                      1
PBR                      0
BPS                      0
Group_PER                0
Revenue                  0
Operating_Income         0
Net_Income               0
Dividend                61
Debt                     0
Debt_continuous          0
Retention                0
Retention_Continuous     0
Open                     0
High                     0
Low                      0
Close                    0
DaytoDay                 0
Volume                   0
Highest_Price            0
Highest_Date             0
update_date              0
dtype: int64

칼럼이 너무 많아서 보기 불편하다면, null값이 있는 칼럼명만 뽑아 확인해 볼 수 있습니다.  
`sum()`대신 `any()`를 활용하면 됩니다.


In [28]:
df.columns[df.isnull().any()]

Index(['PER', 'ROE', 'Dividend'], dtype='object')

In [29]:
df[['ID', 'Name', 'PER', 'ROE', 'Dividend']].head(5)

,ID,Name,PER,ROE,Dividend
0,20,동화약품,22.82,5.27,1.23
1,40,KR모터스,-8.77,-25.82,NaN
2,50,경방,16.92,2.98,0.91
3,60,메리츠화재,6.13,26.08,1.85
4,70,삼양홀딩스,3.66,14.53,3.05


- PER : "주가수익비율"입니다. `PER = 시가총액/순이익 = 주가/EPS`
- ROE : "자기자본수익률"입니다. `ROE = 순이익/자기자본`
- Dividend : "배당"입니다. 단, 이 데이터에서는 '주당배당금(원)'대신 '시가배당률(%)'입니다.

# 결측값 제거

결측값을 제거하는 방법은 또 크게 3가지로 나눌 수 있습니다.
- 결측값이 있는 행(row)을 제거한다.
- 결측값이 있는 열(column)을 제거한다.

### 1) 결측값이 있는 행 제거

`dropna()`로 결측값이 있는 행을 제거할 수 있습니다.

In [51]:
del_row = df.dropna()
del_row

,ID,Name,Market,Category,Capital,PER,EPS,ROE,PBR,BPS,...,Retention_Continuous,Open,High,Low,Close,DaytoDay,Volume,Highest_Price,Highest_Date,update_date
0,20,동화약품,KOSPI,제약,3282,22.82,204,5.27,1.16,12534,...,5,11400,11750,11400,11750,300,67692,21550,2020-08-11,2022-05-13 7:04
2,50,경방,KOSPI,"섬유,의류,신발,호화품",4208,16.92,155,2.98,0.46,30043,...,5,15100,15350,14900,15350,200,8396,17750,2022-04-26,2022-05-13 6:48
3,60,메리츠화재,KOSPI,손해보험,44269,6.13,1605,26.08,1.52,22115,...,4,37250,37350,34650,36700,350,545472,53500,2022-01-21,2022-05-13 7:05
4,70,삼양홀딩스,KOSPI,식품,6988,3.66,1645,14.53,0.43,226314,...,1,81800,82300,81000,81600,500,9234,134000,2021-04-22,2022-05-13 6:48
5,75,삼양홀딩스우,KOSPI,식품,190,3.66,1645,14.53,0.43,226314,...,1,62600,62600,61900,62500,200,63,80700,2021-04-20,2022-05-13 6:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,4910,조광페인트,KOSPI,건축자재,1014,-4.60,-1337,-12.94,0.51,17256,...,1,7700,8040,7690,7920,110,26796,11300,2022-01-20,2022-05-13 6:59
246,4960,한신공영,KOSPI,건설,1799,3.75,2254,7.55,0.27,58000,...,5,15800,16050,15550,15550,200,67490,27500,2021-05-10,2022-05-13 6:39
247,4970,신라교역,KOSPI,식품,2016,4.82,556,8.32,0.38,35004,...,5,12500,12600,12400,12600,50,8071,15450,2021-06-11,2022-05-13 6:39
248,4980,성신양회,KOSPI,건축자재,2917,64.89,619,1.57,0.99,16637,...,4,12050,12300,11800,11900,250,256562,20200,2021-12-20,2022-05-13 7:16


다만 이 경우, 모든 칼럼에서 하나의 결측값이라도 있는 행은 모두 사라집니다.  
전체 250개 행밖에 되지 않는 작디작은 데이터였는데 그 중 거의 1/4가 사라져버렸군요.  

칼럼별로 다르게 대처할 필요가 있어보입니다.  
각각 결측값이 하나밖에 없는 `PER`, `ROE`칼럼에서만 결측값이 있는 행을 살펴보겠습니다.

In [52]:
df[df['PER'].isnull() | df['ROE'].isnull()]

,ID,Name,Market,Category,Capital,PER,EPS,ROE,PBR,BPS,...,Retention_Continuous,Open,High,Low,Close,DaytoDay,Volume,Highest_Price,Highest_Date,update_date
223,4440,삼일씨엔에스,KOSPI,건축자재,1198,NaN,183,NaN,0.43,21552,...,3,9490,9740,9340,9410,80,19282,16200,2021-04-08,2022-05-13 6:56


운이 좋은건지 나쁜건지 같은 행에서 결측이 발생했군요! 행 하나만 제거하면 두 칼럼이 행복해질 것 같습니다.

저 행을 직접 제거해도 되겠지만 '결측값처리'라는 주제에 맞게 `dropna()`를 이용하려고 합니다.
결측값을 탐색하려는 칼럼들만 `subset`으로 지정해서 사용하면 해결입니다.

In [59]:
del_row = df.dropna(subset=['PER','ROE'])
del_row

,ID,Name,Market,Category,Capital,PER,EPS,ROE,PBR,BPS,...,Retention_Continuous,Open,High,Low,Close,DaytoDay,Volume,Highest_Price,Highest_Date,update_date
0,20,동화약품,KOSPI,제약,3282,22.82,204,5.27,1.16,12534,...,5,11400,11750,11400,11750,300,67692,21550,2020-08-11,2022-05-13 7:04
1,40,KR모터스,KOSPI,자동차,761,-8.77,-48,-25.82,2.25,385,...,1,786,804,786,792,8,80163,1450,2019-07-05,2022-05-13 6:31
2,50,경방,KOSPI,"섬유,의류,신발,호화품",4208,16.92,155,2.98,0.46,30043,...,5,15100,15350,14900,15350,200,8396,17750,2022-04-26,2022-05-13 6:48
3,60,메리츠화재,KOSPI,손해보험,44269,6.13,1605,26.08,1.52,22115,...,4,37250,37350,34650,36700,350,545472,53500,2022-01-21,2022-05-13 7:05
4,70,삼양홀딩스,KOSPI,식품,6988,3.66,1645,14.53,0.43,226314,...,1,81800,82300,81000,81600,500,9234,134000,2021-04-22,2022-05-13 6:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,4920,씨아이테크,KOSPI,컴퓨터와주변기기,429,-8.51,119,-15.91,1.27,1202,...,2,1070,1115,1065,1075,5,272694,2035,2021-04-27,2022-05-13 6:59
246,4960,한신공영,KOSPI,건설,1799,3.75,2254,7.55,0.27,58000,...,5,15800,16050,15550,15550,200,67490,27500,2021-05-10,2022-05-13 6:39
247,4970,신라교역,KOSPI,식품,2016,4.82,556,8.32,0.38,35004,...,5,12500,12600,12400,12600,50,8071,15450,2021-06-11,2022-05-13 6:39
248,4980,성신양회,KOSPI,건축자재,2917,64.89,619,1.57,0.99,16637,...,4,12050,12300,11800,11900,250,256562,20200,2021-12-20,2022-05-13 7:16
